<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/046_README_GeneratorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## ✅ README Generator Agent (Using the GAME Framework)

This lecture demonstrates how to build a real-world agent that analyzes a Python project and generates a `README.md` file—all by applying the **GAME architecture** (Goals, Actions, Memory, Environment).

---

### 🎯 Agent Purpose

> **Generate a README file by reading and understanding Python files in the project.**

---

### 🧱 Components Overview

#### 1. **Goals**

Define the agent’s objectives:

```python
goals = [
    Goal(priority=1, name="Gather Information", description="Read each file in the project"),
    Goal(priority=1, name="Terminate", description="Call the terminate action when done and include README content")
]
```

#### 2. **Actions**

Defined using the `Action` class:

* `list_project_files()`: Lists all `.py` files in the directory.
* `read_project_file(name)`: Reads the contents of a given file.
* `terminate(message)`: Ends the agent and outputs the README content.

Each action has:

* a clear name
* a function
* a parameter schema
* a `terminal` flag to stop the loop

#### 3. **AgentLanguage**

Uses `AgentFunctionCallingActionLanguage()` to:

* Format goals/actions for the LLM
* Use OpenAI function calling
* Parse tool responses

#### 4. **Environment**

Basic environment using local file system access, no custom execution needed here:

```python
environment = Environment()
```

#### 5. **Agent Construction**

All pieces are combined in a single modular `Agent` object:

```python
agent = Agent(goals, agent_language, action_registry, generate_response, environment)
```

---

### 🔁 Agent Execution Flow

1. **User Prompt** → `"Write a README for this project."`
2. Agent chooses to:

   * Call `list_project_files`
   * Read each file one by one
   * Accumulate file contents in memory
3. Agent eventually calls `terminate()` and returns a complete README string

---

### 💡 Why This Matters

This agent:

* Demonstrates **modular design**
* Requires no change to the main loop to extend capabilities
* Makes it easy to swap out LLMs, environments, or actions

---

### 🚀 What You Can Build Next

You can now extend this agent to:

* Support `Jupyter` or `Markdown` files
* Run inside `GitHub Actions`
* Use custom environments for remote file reads
* Leverage memory for multi-step reasoning across files




In [ ]:
def main():
    # Define the agent's goals
    goals = [
        Goal(priority=1, name="Gather Information", description="Read each file in the project"),
        Goal(priority=1, name="Terminate", description="Call the terminate call when you have read all the files "
                                                       "and provide the content of the README in the terminate message")
    ]

    # Define the agent's language
    agent_language = AgentFunctionCallingActionLanguage()

    def read_project_file(name: str) -> str:
        with open(name, "r") as f:
            return f.read()

    def list_project_files() -> List[str]:
        return sorted([file for file in os.listdir(".") if file.endswith(".py")])

    # Define the action registry and register some actions
    action_registry = ActionRegistry()
    action_registry.register(Action(
        name="list_project_files",
        function=list_project_files,
        description="Lists all files in the project.",
        parameters={},
        terminal=False
    ))
    action_registry.register(Action(
        name="read_project_file",
        function=read_project_file,
        description="Reads a file from the project.",
        parameters={
            "type": "object",
            "properties": {
                "name": {"type": "string"}
            },
            "required": ["name"]
        },
        terminal=False
    ))
    action_registry.register(Action(
        name="terminate",
        function=lambda message: f"{message}\nTerminating...",
        description="Terminates the session and prints the message to the user.",
        parameters={
            "type": "object",
            "properties": {
                "message": {"type": "string"}
            },
            "required": []
        },
        terminal=True
    ))

    # Define the environment
    environment = Environment()

    # Create an agent instance
    agent = Agent(goals, agent_language, action_registry, generate_response, environment)

    # Run the agent with user input
    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)

    # Print the final memory
    print(final_memory.get_memories())


if __name__ == "__main__":
    main()



### 🧠 1. This Is a Full-Stack Agent (In Miniature)

This agent brings together every major component you’ve learned:

* 🎯 Goals: clear intent and prioritization
* 🛠️ Actions: well-scoped, with clean schemas
* 🧠 Memory: tracking previous reads and results
* 🧱 AgentLanguage: OpenAI function calling
* 🌍 Environment: simple local file access

It’s a perfect end-to-end example of how all the parts work in unison. It’s also a great **template** for other project agents.

---

### 🧩 2. Goal-Driven Behavior Reduces Prompt Complexity

Instead of relying on a giant prompt template, the agent breaks things down into **goals and tools**, letting the LLM figure out the execution order. This is powerful because:

* The LLM stays focused on *why* and *what* instead of *how*.
* You reduce brittle prompt engineering.
* The loop keeps memory and context tightly scoped.

---

### ⚒️ 3. Small Tools, Big Results

The agent only needs three simple tools:

* `list_project_files()`
* `read_project_file(file_name)`
* `terminate(message)`

This highlights how **you don’t need a huge toolkit**—you just need the right abstractions and structure to let the LLM orchestrate.

---

### 🧰 4. Extending It Is Easy

Because the agent is modular, you can imagine adding:

* `summarize_code()`
* `extract_dependencies()`
* `generate_install_instructions()`
* `create_usage_examples()`

Each one would be a new action with a schema, and the core logic wouldn’t need to change at all. This is a **major advantage** over procedural or hardcoded agents.

---

### 🧼 5. Termination Is Part of the Conversation

The use of a `terminate()` tool ensures that the agent **knows when it’s done** and can wrap up with a helpful output. It also acts as a structured way to signal completion—very helpful in loops.

---

### 📦 6. Practical Takeaway

If you ever need to explain how agent design works in an interview or on your LinkedIn, this is the agent to showcase. It’s:

* Concrete
* Useful
* Modular
* Easy to explain






## 🧠 The Core Idea: Let the LLM Decide *How*

In traditional LLM prompting, we often write large prompt templates that:

* Outline **every step** the LLM should follow
* Include **multi-step examples** ("few-shot")
* Repeat instructions to try to control behavior

This is **brittle**:

* If the task slightly changes, the prompt breaks
* LLMs can get confused when too many steps are hardcoded
* Prompts grow huge and expensive to process

This point is *fundamental* to modern agent design, and understanding it deeply will make you a better system designer and collaborator with LLMs.

---

## ✅ With Goal-Driven Agents:

You tell the LLM:

* “Here’s what you’re trying to do” (*Goals*)
* “Here are the tools you can use to do it” (*Actions*)
* “Here’s what you’ve already seen or done” (*Memory*)
* “I’ll take care of execution—you just tell me what to do next” (*Agent Loop*)

### This radically simplifies prompting:

* No more giant templates
* No more overfitting behavior through prompt tweaks
* You treat the LLM like a *planner*, not a parser or script follower

---

### 🎯 Example Comparison

#### Traditional Prompt Approach

```
You are an assistant that generates README files.

Step 1: List the files.
Step 2: Read the main.py file.
Step 3: Generate a README in markdown format.
...
```

⬇️ vs.

#### Agent Approach (GAME Style)

```
Goals:
- Understand the structure of this Python project
- Read the main modules
- Generate a concise and clear README.md file

Available Tools:
- list_project_files()
- read_project_file(file_name)
- terminate(message)
```

The LLM now decides *when* to call which tool, based on what it sees in the memory.

---

## 🧠 Benefits in Practice

| 🔍 Feature                       | ⚙️ Result                             |
| -------------------------------- | ------------------------------------- |
| No step-by-step prompt           | More flexible agent                   |
| Small, reusable goals            | Easily extensible tasks               |
| Schema-bound tools               | LLM output stays predictable          |
| Memory injected only when needed | Smaller token usage, clearer focus    |
| Function-calling + planning      | No parsing errors or ambiguous output |

---

## 🧰 Agent Loop = "Structured Autonomy"

This structure gives you:

* Predictability (because tools are structured)
* Flexibility (because plans are dynamic)
* Control (because execution happens in code, not in text)

You’re no longer relying on *magic prompts* — you’re engineering systems with clear responsibilities.





## ✅ The GOAL-Based Approach *Delegates Prompting to the LLM*

This is one of the most **important mental shifts** in moving from prompt-centric "agent hype" to **robust agent systems**.

### 🔁 Traditional Prompt-Centric Design:

* You (the developer) write one giant prompt.
* You hardcode every instruction.
* You try to anticipate *every edge case and step* in advance.
* When anything changes — new file, new task — you have to edit the whole thing.

This approach is *fragile* and puts the burden of reasoning and formatting on the developer.

---

### 🎯 Goal-Based Agent Design (GAME-style):

* You define the **intent** (goals).
* You define the **capabilities** (actions).
* You give the LLM **structured context** (memory + environment).
* You let the LLM decide *how* to achieve that intent, step-by-step.

Instead of writing a long prompt, you're writing a **system interface** that the LLM uses.

---

### 🔍 What does the LLM see?

You're sending it a structured message that says:

```json
{
  "goals": [
    "Generate a README based on the structure of this repo"
  ],
  "actions": [
    { "name": "list_project_files", "description": "List the files in the directory" },
    { "name": "read_project_file", "parameters": { "file_name": "..." }, "description": "Read a file" },
    { "name": "terminate", "description": "Finish the task with a message" }
  ],
  "memory": [
    // Summary of what the LLM already knows
  ]
}
```

That *is* the prompt — but it’s dynamically generated, clean, and reusable across tasks.

---

## 🧠 Benefits of This Delegation

| Prompt-Centric        | Goal-Based                  |
| --------------------- | --------------------------- |
| Rigid templates       | Dynamic goal interpretation |
| Fragile to changes    | Extensible + adaptable      |
| High developer burden | LLM handles planning        |
| Hard to debug         | Modular and testable        |
| Easy to forget steps  | Agent loop ensures flow     |

---

## 🤖 Final Takeaway

The GOAL isn't to write better prompts.
The GOAL *is the prompt*.

> You’ve moved from **prompt engineer** to **agent system designer**.

